<h1>Speech to Text and Sentiment Analysis on Audio Files Automation
    <h4>Azure Cognitive Servisler içerisinde yer alan Speech servisi ve Text Analytıcs servisi ile örnek bir sesten metne dönüşüm ve duygu analizi scripti.

Congitive Servisler Dökümantasyonu: https://docs.microsoft.com/en-us/azure/cognitive-services/

Bu servislerin kullanımı için Azure Portal üzerinden iki serviş oluşturulmalı, **Speech** ve **Text Analytics**.

**Speech Servisi**

Speech Servisleri Dökümantasyonu: https://docs.microsoft.com/en-us/azure/cognitive-services/speech-service/

Speech to Text Dökümantasyonu: https://docs.microsoft.com/en-us/azure/cognitive-services/speech-service/speech-to-text

**Text Analytics Servisi**

Text Analytics Dökümantasyonu:https://docs.microsoft.com/en-us/azure/cognitive-services/text-analytics/

Entity Recognition Desteklenen Entitiy'ler: https://docs.microsoft.com/en-us/azure/cognitive-services/text-analytics/named-entity-types?tabs=general

In [ ]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
import glob
import azure.cognitiveservices.speech as speechsdk
import time
import json
import pandas as pd
import csv

<h5>Bu örnekte 2 adet ses dosyası ile çalışmaktayız. Bu dosyaların wav formatında olması gerekiyor.
Ses dosyalarının ve output dosyasının ismi aşağıda tanımlanmalı. Output dosyası csv formatında oluşturulmuş olmalı.

In [ ]:
audio_input1 = speechsdk.AudioConfig(filename="audio1.wav") 

output1 = "output1"

In [ ]:
audio_input2 = speechsdk.AudioConfig(filename="audio1.wav") 
output2 = "output2"

<h5>Daha sonra oluşturduğumuz bu servislere ait bilgiler girilmeli.
    Bu bilgilere Resource içerisindeki Keys and Endpoint kısmından ulaşabiliriz.
 

In [ ]:
speech_key = "key"
speech_key = "region"

In [ ]:
textanalytics_key = "key"
textanalytics_endpoint = "endpoint"

<h5>Speech to Text Fonksiyonları

In [ ]:
speech_config = speechsdk.SpeechConfig(subscription= speech_key, region= speech_region) 

In [ ]:
def speech_recognize_continuous_from_file(file, output_file_name):
    
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_key) 
    audio_config = file
    
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config, language="tr-TR")
    
    done = False
    def stop_cb(evt):
        #print('CLOSING on {}'.format(evt))
        speech_recognizer.stop_continuous_recognition()
        nonlocal done
        done = True
    all_results = []
    def handle_final_result(evt):
        all_results.append(evt.result.text)
    speech_recognizer.recognized.connect(handle_final_result)
    
    #Anlik speech to Text donusumunu takip etmek icin asagidaki satirlari uncomment ediniz.
    
    #speech_recognizer.recognizing.connect(lambda evt: print('RECOGNIZING: {}'.format(evt)))
    #speech_recognizer.recognized.connect(lambda evt: print('RECOGNIZED: {}'.format(evt)))
    #speech_recognizer.session_started.connect(lambda evt: print('SESSION STARTED: {}'.format(evt)))
    #speech_recognizer.session_stopped.connect(lambda evt: print('SESSION STOPPED {}'.format(evt)))
    ##speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))
    
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)
    
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)
    print("Printing all results:")
    print(all_results)
    df = pd.DataFrame(all_results)
    df
    file_name = output_file_name + r".csv"
    df.to_csv(file_name)
    print ("Audio File: converted successfully")
    return all_results

In [ ]:
def speech_to_text(audio_input, output_file_name):
    print ("PROGRAM START")
    tic = time.perf_counter()
    
    file= audio_input

    # Loop to call function to convert audio files to text
    #run_speech_to_text_small_audio_files(file)
    all_results = speech_recognize_continuous_from_file(file, output_file_name)
    print(file)
    print ("PROGRAM END")
    toc = time.perf_counter()
    print(f"Converted speech to text in {toc - tic:0.4f} seconds")
    return all_results

<h5>Text Analytics Fonksiyonları

In [ ]:
key = textanalytics_key
endpoint = textanalytics_endpoint
def authenticateClient():
    credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
        endpoint=endpoint, credential=credential)
    return text_analytics_client
client = authenticateClient()

In [ ]:
def sentiment_analysis_example(client, sentence):
    documents = [sentence]
    response = client.analyze_sentiment(documents=documents)[0]
    print("\tDocument Sentiment: {}".format(response.sentiment))
    print("\tOverall scores: positive={0:.2f}; neutral={1:.2f}; negative={2:.2f} ".format(
        response.confidence_scores.positive,
        response.confidence_scores.neutral,
        response.confidence_scores.negative,
    ))
    for idx, sentence in enumerate(response.sentences):
        print("\t\tSentence: {}".format(sentence.text))
        print("\t\tSentence {} sentiment: {}".format(idx+1, sentence.sentiment))
        print("\t\tSentence score:\tPositive={0:.2f}\tNeutral={1:.2f}\tNegative={2:.2f}\n".format(
            sentence.confidence_scores.positive,
            sentence.confidence_scores.neutral,
            sentence.confidence_scores.negative,
        ))
    

    return (
        response.confidence_scores.positive,
        response.confidence_scores.neutral,
        response.confidence_scores.negative,
    )

In [ ]:
def entity_recognition_example(client, sentence):
    try:
        documents = [{"language": "tr","id": "1","text": sentence }]
        #documents = [sentence]
        result = client.recognize_entities(documents = documents)[0]

        print("\n\tNamed Entities:\n")
        for entity in result.entities:
            print("\t\tText: \t", entity.text, "\tCategory: \t", entity.category, "\tSubCategory: \t", entity.subcategory,
                    "\n\t\tConfidence Score: \t", round(entity.confidence_score, 2), "\tLength: \t", entity.length, "\tOffset: \t", entity.offset, "\n")
            
        return result
    except Exception as err:
        print("\tEncountered exception. {}".format(err))

<h3>AUDIO FILE 1
<h5>Speech To Text Dönüşümü

In [ ]:
stt_results1 = speech_to_text(audio_input1, output1)

<h3>AUDIO FILE 1
<h5>Sentiment Analizi, Key Extraction'u ve Entity Bulma

In [ ]:
num_of_sentences = len(stt_results1)
#print(num_of_sentences)
all_results = []
for i in range (num_of_sentences):
    sentence = stt_results1[i]
    if sentence != '':
        print(i+1,"- ", sentence)
        rp, rne, rn = sentiment_analysis_example(client, sentence)
        all_results.append([sentence, rp, rne, rn])
        entity_recognition_example(client, sentence)
df = pd.DataFrame(all_results)
df
output_file_name = "output1-ta"
file_name = output_file_name + r".csv"
df.to_csv(file_name)

***********************************************************

<h3>AUDIO FILE 2
<h5>Speech To Text Dönüşümü

In [ ]:
stt_results2 = speech_to_text(audio_input2, output2)

<h3>AUDIO FILE 2
<h5>Sentiment Analizi ve Entity Bulma

In [ ]:
num_of_sentences = len(stt_results2)
#print(num_of_sentences)
for i in range (num_of_sentences):
    sentence = stt_results2[i]
    if sentence != '':
        print(i+1,"- ", sentence)
        rp, rne, rn = sentiment_analysis_example(client, sentence)
        all_results.append([sentence, rp, rne, rn])
        entity_recognition_example(client, sentence)
df = pd.DataFrame(all_results)
df
output_file_name = "output2-ta"
file_name = output_file_name + r".csv"
df.to_csv(file_name)